# Задание по нефтегазу, методика СРК

### Импорт библиотек

In [17]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from numpy.linalg import norm
from scipy.optimize import fsolve
#import solver

### Здесь предтавлены константы:

In [18]:
T = 273.15 # температура смеси
p = 5.0
N = 4 # номер состава
path = 'C:\\Users\\timur\\Desktop\\курс по нефтегазу\\data.xlsx'
xls = pd.ExcelFile(path)
coeff = pd.read_excel(xls, 'Коэффициенты')
compos = pd.read_excel(xls, 'Составы')
del compos['сумма']
pair_inter_coeff = pd.read_excel(xls, 'Коэф. пар. взаим.', header=None)
pair_inter_coeff.dropna(axis='columns',how='all', inplace=True)

In [20]:
compos

,метан,этан,пропан,изобутан,бутан,пентан,гептан,декан,азот,углекислый газ,сероводород
0,0.78120,0.03000,0.02000,0.0048,0.0010,0.00000,0.00000,0.00000,0.16300,0.00000,0.00000
1,0.85330,0.04230,0.02640,0.0060,0.0056,0.03440,0.00000,0.00000,0.02660,0.00540,0.00000
2,0.72270,0.04551,0.02474,0.0000,0.0000,0.05205,0.03650,0.02814,0.03020,0.03015,0.03001
3,0.80324,0.06024,0.03066,0.0000,0.0000,0.04381,0.03108,0.02431,0.00478,0.00188,0.00000
4,0.77430,0.05740,0.02990,0.0000,0.0000,0.04660,0.03590,0.02630,0.00000,0.00000,0.02960
5,0.80970,0.05660,0.03060,0.0000,0.0000,0.04570,0.03300,0.02440,0.00000,0.00000,0.00000
6,0.76430,0.07460,0.03120,0.0059,0.0121,0.01090,0.00790,0.06700,0.00120,0.02490,0.00000


In [21]:
coeff

,Вещество,Критическое давление,Критическая температура,Ацентрический фактор
0,метан,4.5992,190.56,0.01142
1,этан,4.8722,305.32,0.09950
2,пропан,4.2512,369.89,0.15210
3,изобутан,3.6290,407.81,0.18400
4,бутан,3.7960,425.13,0.20100
5,пентан,3.3700,469.60,0.25100
6,гептан,2.7360,540.13,0.34900
7,декан,2.1030,617.70,0.48840
8,азот,3.3958,126.19,0.03720
9,углекислый газ,7.3773,304.13,0.22394


In [19]:
pair_inter_coeff

,0,1,2,3,4,5,6,7,8,9,10
0,0.000,0.005,0.010,0.010,0.010,0.030,0.035,0.045,0.025,0.105,0.070
1,0.005,0.000,0.005,0.010,0.010,0.010,0.020,0.020,0.010,0.130,0.085
2,0.010,0.005,0.000,0.000,0.000,0.020,0.005,0.005,0.090,0.125,0.080
3,0.010,0.010,0.000,0.000,0.000,0.005,0.005,0.005,0.095,0.115,0.075
4,0.010,0.010,0.000,0.000,0.000,0.005,0.005,0.005,0.095,0.115,0.075
5,0.030,0.010,0.020,0.005,0.005,0.000,0.000,0.000,0.100,0.115,0.070
6,0.035,0.020,0.005,0.005,0.005,0.000,0.000,0.000,0.115,0.115,0.060
7,0.045,0.020,0.005,0.005,0.005,0.000,0.000,0.000,0.125,0.115,0.055
8,0.025,0.010,0.090,0.095,0.095,0.100,0.115,0.125,0.000,0.000,0.130
9,0.105,0.130,0.125,0.115,0.115,0.115,0.115,0.115,0.000,0.000,0.135


In [16]:
z = np.asarray(compos.iloc[N].to_list(), dtype=float)
p_c = np.asarray(coeff['Критическое давление'].to_list(), dtype=float)
T_c = np.asarray(coeff['Критическая температура'].to_list(), dtype=float)

In [18]:
p_r = p / p_c
T_r = T / T_c
omega = np.asarray(coeff['Ацентрический фактор'].to_list(), dtype=float)
m = np.asarray([0.48 + 1.574 * omega_i - 0.176 * omega_i * omega_i for omega_i in omega])
alpha = np.asarray([(1 + m[i] * (1 - np.sqrt(T_r[i]))) for i in range(len(T_r))])
K = np.asarray([np.exp(5.37 * (1 + omega[i]) * (1 - 1 / T_r[i])) / p_r[i] for i in range(len(T_r))])

In [30]:
def rashford(z, K):
    
    def inner(v):
        sum = 0
        for i in range(len(z)):
            sum += z[i] * (K[i] - 1) / (v * (K[i] - 1) + 1)
        return sum
    return inner      


def Z(A, B):
    
    def inner(x):
        return x ** 3 - x ** 2 + (A - B - B ** 2) * x - A * B
    return inner

In [22]:
V = solver.solve(0, 1, rashford(z, K))
y = np.asarray([z[i] * K[i] / (V * (K[i] - 1) + 1) for i in range(len(z))], dtype=float)
x = np.asarray([y[i] / K[i] for i in range(len(y))])

NameError: name 'solver' is not defined

In [23]:
B_p = np.asarray([0.08664 * p_r[i] / T_r[i] for i in range(len(p_r))])
A_p = np.asarray([0.42747 * alpha[i] * p_r[i] / (T_r[i] ** 2) for i in range(len(T_r))])
A_v, B_v, A_l, B_l = 0, 0, 0, 0
for i in range(len(x)):
    B_v += y[i] * 0.08664 * p_r[i] / T_r[i]
    B_l += x[i] * 0.08664 * p_r[i] / T_r[i]
    for j in range(len(x)):
        A_v += y[i] * y[j] * np.sqrt((0.42747 ** 2) * alpha[i] * alpha[j] * p_r[i] * p_r[j] / ((T_r[i] ** 2) * (T_r[j] ** 2)))
        A_l += x[i] * x[j] * np.sqrt((0.42747 ** 2) * alpha[i] * alpha[j] * p_r[i] * p_r[j] / ((T_r[i] ** 2) * (T_r[j] ** 2)))

NameError: name 'x' is not defined

In [ ]:
Z_v = np.max(fsolve(Z(A_v, B_v), 0))
Z_l = np.min(fsolve(Z(A_l, B_l), 0))

In [ ]:
Fi_v = np.asarray([(Z_v - 1) * (B_p[i] / B_v) - np.log(Z_v - B_v) - (A_v / B_v) * (2 * np.sqrt(A_p[i] / A_v) - B_p[i] / B_v) * np.log((Z_v + B_v) / Z_v) for i in range(B_p)])
Fi_l = np.asarray([(Z_l - 1) * (B_p[i] / B_l) - np.log(Z_l - B_l) - (A_l / B_l) * (2 * np.sqrt(A_p[i] / A_l) - B_p[i] / B_l) * np.log((Z_l + B_l) / Z_l) for i in range(B_p)])
K_next = np.asarray([Fi_l[i] / Fi_v[i] for i in range(len(Fi_v))])

In [ ]:
while norm(K_next - K) / norm(K) >= 10 ** (-5):
    K = K_next
    V = solver.solve(0, 1, rashford(z, K))
    y = np.asarray([z[i] * K[i] / (V * (K[i] - 1) + 1) for i in range(len(z))], dtype=float)
    x = np.asarray([y[i] / K[i] for i in range(len(y))])
    A_v, B_v, A_l, B_l = 0, 0, 0, 0
    for i in range(len(x)):
        B_v += y[i] * 0.08664 * p_r[i] / T_r[i]
        B_l += x[i] * 0.08664 * p_r[i] / T_r[i]
        for j in range(len(x)):
            A_v += y[i] * y[j] * np.sqrt((0.42747 ** 2) * alpha[i] * alpha[j] * p_r[i] * p_r[j] / ((T_r[i] ** 2) * (T_r[j] ** 2)))
            A_l += x[i] * x[j] * np.sqrt((0.42747 ** 2) * alpha[i] * alpha[j] * p_r[i] * p_r[j] / ((T_r[i] ** 2) * (T_r[j] ** 2)))
    Z_v = np.max(fsolve(Z(A_v, B_v), 0))
    Z_l = np.min(fsolve(Z(A_l, B_l), 0))
    Fi_v = np.asarray([(Z_v - 1) * (B_p[i] / B_v) - np.log(Z_v - B_v) - (A_v / B_v) * (2 * np.sqrt(A_p[i] / A_v) - B_p[i] / B_v) * np.log((Z_v + B_v) / Z_v) for i in range(B_p)])
    Fi_l = np.asarray([(Z_l - 1) * (B_p[i] / B_l) - np.log(Z_l - B_l) - (A_l / B_l) * (2 * np.sqrt(A_p[i] / A_l) - B_p[i] / B_l) * np.log((Z_l + B_l) / Z_l) for i in range(B_p)])
    K_next = np.asarray([Fi_l[i] / Fi_v[i] for i in range(len(Fi_v))])